**2단계:**

- 유투브 AI 내용 검색 Agent 추가
    - 유투브 API 활용
    - 예시 입력 프롬프트 : AI 뉴스관련 영상을 알려줘
    - Agent를 통해서 유투브 API를 통해 뉴스 검색 후 검색 된 내용을 프롬프트에 넣어서 알려주기
        - streamlit 유투브 영상 화면 까지 같이 보여주기 (재생할 수 있는)
    - Agent는 유저의 쿼리를 LLM이 분석해서 특정 함수를 실행
    - **공식문서 참고해서 Agent내용 학습후 코드화 (베스트)**
        - Quick Start (가이드 코드)
        - Lanchain : 안배웠다하더라도 스스로 찾아서 할 수 있어야한다.
        - App

### 아이디어 정리
1. 사용자의 질문 입력
2. 질문을 LLM으로 보내 검색 키워드 분석
3. 검색 키워드에 맞는 tool 설정
4. tool 동작 수행

In [5]:
import os
import googleapiclient.discovery
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain.agents import Tool
from NewsRAG import AINewsRAG

In [6]:
# 환경 변수 로드 
load_dotenv()

# 환경 변수에서 경로 가져오기
vector_store_path = "../ai_news_vectorstore"

# 임베딩 모델 초기화 
embedding_model = OpenAIEmbeddings(
    model=os.getenv("OPENAI_EMBEDDING_MODEL")
)

# 환경 변수에서 OpenAI API 키를 불러오기
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("OPENAI_API_KEY 환경 변수를 설정해주세요.")

# 환경 변수에서 YouTube API 키를 불러오기
youtube_api_key = os.getenv("YOUTUBE_API_KEY")
if not youtube_api_key:
    print("YOUTUBE_API_KEY 환경 변수를 설정해주세요.")

# RAG 시스템 초기화
rag = AINewsRAG(embedding_model)

try:
    # 기존 벡터 스토어 로드 시도
    rag.load_vector_store(vector_store_path)
    print("✅ 기존 벡터 스토어를 로드했습니다.")
    
except Exception as e:
    print(f"벡터 스토어 로드 실패: {str(e)}")

2024-12-18 00:31:15,649 - 벡터 스토어를 ../ai_news_vectorstore에서 로드합니다...
2024-12-18 00:31:16,294 - 벡터 스토어 로드가 완료되었습니다.
✅ 기존 벡터 스토어를 로드했습니다.


In [14]:
@tool
def search_news(query: str, k: int = 5):
    """
    AI 뉴스를 검색합니다.
    """
    while True:
        query = query.strip()

        if not query:
            continue
            
        if query.lower() in ['q', 'quit']:
            print("\n👋 검색을 종료합니다.")
            break

        try:
            print(f"\n'{query}' 검색을 시작합니다...")
            
            results = rag.search(query, k=k)
            
            print(f"\n✨ 검색 완료! {len(results)}개의 결과를 찾았습니다.\n")
            
            # 결과 출력
            for i, doc in enumerate(results):
                print(f"\n{'='*80}")
                print(f"검색 결과 {i}/{len(results)}")
                print(f"제목: {doc.metadata['title']}")
                print(f"날짜: {doc.metadata['date']}")
                print(f"URL: {doc.metadata['url']}")
                print(f"{'-'*40}")
                print(f"내용:\n{doc.page_content[:300]}...")
            
            # 종료
            break
        
        except Exception as e:
            print(f"\n❌ 검색 중 오류가 발생했습니다: {str(e)}")

In [15]:
@tool
def search_video(query, max_results=5):
        """
        YouTube API를 사용하여 검색.
        """
        youtube = googleapiclient.discovery.build(
            "youtube", "v3", developerKey=youtube_api_key
        )

        request = youtube.search().list(
            part="snippet",
            q=query,
            type="video",
            maxResults=max_results
        )
        response = request.execute()

        results = [
            {
                "title": item["snippet"]["title"],
                "description": item["snippet"]["description"],
                "video_id": item["id"]["videoId"]
            }
            for item in response.get("items", [])
        ]
        return results

In [16]:
tools = [
    Tool(
        name="Search youtube tool",
        func=search_video,
        description="YouTube API를 사용하여 검색합니다."
    ),
    Tool(
        name="Search news tool",
        func=search_news,
        description="AI 뉴스를 검색합니다."
    )
]

In [17]:
class SearchResult(BaseModel):
    """
    사용자 질문: str
    액션: str
    검색 키워드: str
    tool 설정: str
    """
    user_query: str
    action: str
    search_keywords: str
    tool: str

In [18]:
class AIAgent:
    def __init__(self, openai_api_key, youtube_api_key, llm_model="gpt-4o"):
        self.openai_api_key = openai_api_key
        self.youtube_api_key = youtube_api_key
        self.llm_model = llm_model
    
    def analyze_query(self, user_query):
        """
        LLM을 사용하여 유저 쿼리를 분석하고 그 결과를 반환.
        """
        llm = ChatOpenAI(
            model=self.llm_model,
            temperature=0.1,
            api_key=self.openai_api_key,
        )
        
        self.output_parser = PydanticOutputParser(
            pydantic_object=SearchResult
        )
        
        self.prompt = PromptTemplate(
            input_variables=["user_query"],
            partial_variables={
                "format_instructions": self.output_parser.get_format_instructions()
            },
            template=
            """
            당신은 AI 관련 정보를 제공하는 도우미입니다.
            먼저 입력된 질의가 AI 관련 내용인지 확인하세요.

            AI 관련 주제 판단 기준:
            - AI 기술 (머신러닝, 딥러닝, 자연어처리 등)
            - AI 도구 및 서비스 (ChatGPT, DALL-E, Stable Diffusion 등)
            - AI 회사 및 연구소 소식
            - AI 정책 및 규제
            - AI 교육 및 학습
            - AI 윤리 및 영향

            AI 관련 질의가 아닌 경우:
            - action을 "not_supported"로 설정
            - search_keyword는 빈 문자열로 설정            

            AI 관련 질의인 경우 다음 작업을 수행하세요:
            1. 검색 도구 선정: 질의 의도 분석 기반 최적 도구 선택
            2. 키워드 추출: 최적화 검색어 생성

            사용 가능한 도구:
            1. search_video: AI 관련 영상 콘텐츠 검색 특화
            2. search_news: AI 관련 뉴스 및 기사 검색 특화

            도구 선택 기준:
            A) search_video 선정 조건:
            - 영상 콘텐츠 요구 (영상, 동영상)
            - 교육 자료 요청 (강의, 강좌, 수업)
            - 실습 가이드 (튜토리얼, 가이드, 설명)
            - 시각적 설명 (시연, 데모)

            B) search_news 선정 조건:
            - 뉴스 콘텐츠 (뉴스, 소식)
            - 기사 요청 (기사, 글)
            - 정보 탐색 (정보, 현황, 동향)
            - 연구 자료 (연구, 조사, 분석)

            키워드 추출 규칙:
            1. 핵심 주제어 분리
            - AI 관련 핵심 개념 추출
            - 매체 유형 지시어 제거 (정보, 뉴스, 영상, 기사 등)
            - 보조어 및 조사 제거

            2. 의미론적 최적화
            - 전문 용어 완전성 유지
            - 개념 간 관계성 보존
            - 맥락 적합성 확보

            분석 대상 질의: {user_query}

            {format_instructions}
            """,
        )

        # 실행 체인 생성 - 프롬프트 처리부터 결과 파싱까지의 전체 흐름
        self.chain = RunnableSequence(
            first= {"user_query": RunnablePassthrough()} | self.prompt,  # 먼저 프롬프트 처리
            middle=[llm],  # 그 다음 LLM으로 처리
            last=self.output_parser,  # 마지막으로 결과 파싱
        )
        
        response = self.chain.invoke(user_query)  # 질문 분석
        print(response)
        
        return response.model_dump()  # json 형식으로 변형형

    def format_results_for_display(self, results):
        """
        검색 결과를 스트림릿에서 보여줄 수 있도록 포맷팅.
        """
        for result in results:
            print(f"### {result['title']}")
            if result['video_id']:
                video_url = f"https://www.youtube.com/watch?v={result['video_id']}"
                print(video_url)

In [19]:
def main():
    try:
        # 유저 입력 받기
        print("AI Search Agent")
        user_query = input("검색할 내용을 입력하세요 (예: AI 뉴스 관련 영상을 알려줘):")

        if user_query:
            # Agent 초기화
            agent = AIAgent(openai_api_key, youtube_api_key)
            
            # 쿼리 분석
            print("="*30)
            print("LLM을 통해 입력 쿼리를 분석 중입니다...")
            result = agent.analyze_query(user_query)
            print(f"검색 결과: {result}")
            
            # tool에 따른 동작 실행 
            tool_names = [tool.func.name for tool in tools]  # tool 이름 받기 
            if result['tool'] in tool_names:
                # YouTube 검색
                if result['tool'] == 'search_video':
                    print("="*30)
                    print("YouTube에서 검색 중입니다...")
                    search_results = search_video(result['search_keywords'])

                    # 검색 결과 표시
                    if search_results:
                        print("검색 결과:")
                        agent.format_results_for_display(search_results)
                    else:
                        print("검색 결과가 없습니다.")
            
                # 뉴스 검색
                else:
                    print("="*30)
                    print("뉴스에서 검색 중입니다...")
                    search_results = search_news(result['search_keywords'])

                    # 검색 결과 표시
                    if search_results:
                        print("검색 결과:")
                        agent.format_results_for_display(search_results)
                    else:
                        print("검색 결과가 없습니다.")
            
            else:
                print("AI와 관련된 질문만 받을 수 있습니다.")
            
    except KeyboardInterrupt:
        print("Shutting down process...")
    
    except Exception as e:
        print(f"Error occurred: {e}")

if __name__ == "__main__":
    main()

AI Search Agent
LLM을 통해 입력 쿼리를 분석 중입니다...
user_query='구글 AI에 대한 최근 소식' action='search_news' search_keywords='구글 AI 최근 소식' tool='search_news'
검색 결과: {'user_query': '구글 AI에 대한 최근 소식', 'action': 'search_news', 'search_keywords': '구글 AI 최근 소식', 'tool': 'search_news'}
뉴스에서 검색 중입니다...

'구글 AI 최근 소식' 검색을 시작합니다...
2024-12-18 00:39:54,904 - '구글 AI 최근 소식' 검색을 시작합니다...
2024-12-18 00:39:55,221 - 5개의 관련 문서를 찾았습니다.

✨ 검색 완료! 5개의 결과를 찾았습니다.


검색 결과 0/5
제목: 구글, 긴 기사 요약 등 생성 AI 기반 검색 ‘SGE’ 업데이트
날짜: 2023.08.16 18:03
URL: https://www.aitimes.com/news/articleView.html?idxno=152907
----------------------------------------
내용:
구글, 긴 기사 요약 등 생성 AI 기반 검색 ‘SGE’ 업데이트
 테크크런치는 15일(현지시간) 구글이 인공지능(AI) 기반 ‘검색 생성 경험(SGE)’의 새로운 기능을 데스크톱 버전 크롬과 안드로이드 및 iOS 구글 앱에 도입한다고 보도했다.
SGE는 마이크로소프트(MS)의 '빙 챗'에 대응하는 챗봇 결합형 검색 서비스로, 현재는 일부 신청자를 대상으로 테스트 중이다.
이에 따르면 새로운 기능은 3가지다. 가장 눈에 띄는 것은 AI가 긴 기사를 요약해 주는 ‘검색 중 SGE’라는 기능이다.
검색 결과 웹 페이지 하단의 '생성(G...

검색 결과 1/5
제목: 구글이 만든 AI 음성 비서...목소리를 들어봤더니?
날짜: 2024.08.15 07:00
URL: https://www.